# Gold Meta-Model Training - Attempt 15

**Architecture:** Ridge + XGBoost Weighted Average Ensemble

**Key Changes from Attempt 7:**
1. **Ridge regression** added as second model (with StandardScaler)
2. **Weighted average ensemble:** `w * xgb_pred + (1-w) * ridge_pred`
3. **Weight optimization** on validation set (grid search w in [0.0, 1.0])
4. **Bootstrap ensemble:** 5 XGBoost models + 1 Ridge model

**Inherited from Attempt 7:**
- Same 24 features (including temporal_context_score)
- Same data pipeline, NaN imputation, train/val/test split
- XGBoost Optuna HPO (100 trials, same HP ranges as attempt 7)
- OLS output scaling
- Bootstrap variance-based confidence

**Design:** Based on `docs/design/meta_model_attempt_7.md` with ensemble extension

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
np.random.seed(42)

print(f"XGBoost version: {xgb.__version__}")
print(f"Optuna version: {optuna.__version__}")
print(f"Started: {datetime.now().isoformat()}")

## Feature Definitions

In [ ]:
FEATURE_COLUMNS = [
    # Base features (5)
    'real_rate_change',
    'dxy_change',
    'vix',
    'yield_spread_change',
    'inflation_exp_change',
    # VIX submodel (3)
    'vix_regime_probability',
    'vix_mean_reversion_z',
    'vix_persistence',
    # Technical submodel (3)
    'tech_trend_regime_prob',
    'tech_mean_reversion_z',
    'tech_volatility_regime',
    # Cross-asset submodel (3)
    'xasset_regime_prob',
    'xasset_recession_signal',
    'xasset_divergence',
    # Yield curve submodel (2)
    'yc_spread_velocity_z',
    'yc_curvature_z',
    # ETF flow submodel (3)
    'etf_regime_prob',
    'etf_capital_intensity',
    'etf_pv_divergence',
    # Inflation expectation submodel (3)
    'ie_regime_prob',
    'ie_anchoring_z',
    'ie_gold_sensitivity_z',
    # Options market submodel (1)
    'options_risk_regime_prob',
    # Temporal context submodel (1)
    'temporal_context_score',
]

TARGET = 'gold_return_next'

assert len(FEATURE_COLUMNS) == 24, f"Expected 24 features, got {len(FEATURE_COLUMNS)}"
print(f"Features defined: {len(FEATURE_COLUMNS)} features")

## Data Fetching (API-Based)

In [ ]:
# ============================================================
# API-BASED DATA FETCHING
# ============================================================
print("="*60)
print("FETCHING DATA FROM APIs")
print("="*60)

# === Import libraries ===
import yfinance as yf

# FRED API (install if needed)
try:
    from fredapi import Fred
except ImportError:
    import subprocess
    subprocess.run(["pip", "install", "fredapi"], check=True)
    from fredapi import Fred

# === FRED API key (hardcoded) ===
FRED_API_KEY = "3ffb68facdf6321e180e380c00e909c8"
fred = Fred(api_key=FRED_API_KEY)
print("✓ FRED API initialized")

# === 1. Fetch Gold Price (target) ===
print("\nFetching gold price (GC=F)...")
gold = yf.download('GC=F', start='2014-01-01', end='2026-02-20', progress=False)
gold_df = gold[['Close']].copy()
gold_df.columns = ['gold_price']
gold_df['gold_return'] = gold_df['gold_price'].pct_change() * 100
gold_df['gold_return_next'] = gold_df['gold_return'].shift(-1)  # Next-day return
gold_df = gold_df.dropna(subset=['gold_return_next'])
gold_df.index = pd.to_datetime(gold_df.index).strftime('%Y-%m-%d')
print(f"  Gold: {len(gold_df)} rows")

# === 2. Fetch Base Features ===
print("\nFetching base features...")

# Real Rate (DFII10)
print("  Fetching real rate (DFII10)...")
real_rate = fred.get_series('DFII10', observation_start='2014-01-01')
real_rate_df = real_rate.to_frame('real_rate_real_rate')
real_rate_df.index = pd.to_datetime(real_rate_df.index).strftime('%Y-%m-%d')

# DXY (DX-Y.NYB)
print("  Fetching DXY (DX-Y.NYB)...")
dxy = yf.download('DX-Y.NYB', start='2014-01-01', end='2026-02-20', progress=False)
dxy_df = dxy[['Close']].copy()
dxy_df.columns = ['dxy_dxy']
dxy_df.index = pd.to_datetime(dxy_df.index).strftime('%Y-%m-%d')

# VIX (VIXCLS)
print("  Fetching VIX (VIXCLS)...")
vix = fred.get_series('VIXCLS', observation_start='2014-01-01')
vix_df = vix.to_frame('vix_vix')
vix_df.index = pd.to_datetime(vix_df.index).strftime('%Y-%m-%d')

# Yield Curve (DGS10 - DGS2)
print("  Fetching yield curve (DGS10, DGS2)...")
dgs10 = fred.get_series('DGS10', observation_start='2014-01-01')
dgs2 = fred.get_series('DGS2', observation_start='2014-01-01')
yc_df = pd.DataFrame({'DGS10': dgs10, 'DGS2': dgs2})
yc_df['yield_curve_yield_spread'] = yc_df['DGS10'] - yc_df['DGS2']
yc_df = yc_df[['yield_curve_yield_spread']]
yc_df.index = pd.to_datetime(yc_df.index).strftime('%Y-%m-%d')

# Inflation Expectation (T10YIE)
print("  Fetching inflation expectation (T10YIE)...")
infl_exp = fred.get_series('T10YIE', observation_start='2014-01-01')
infl_exp_df = infl_exp.to_frame('inflation_expectation_inflation_expectation')
infl_exp_df.index = pd.to_datetime(infl_exp_df.index).strftime('%Y-%m-%d')

# Merge base features
base_features = gold_df[['gold_return_next']].copy()
for df in [real_rate_df, dxy_df, vix_df, yc_df, infl_exp_df]:
    base_features = base_features.join(df, how='left')

# Forward-fill missing values (weekends, holidays)
base_features = base_features.ffill()
print(f"  Base features: {len(base_features)} rows, {len(base_features.columns)} columns")

# === 3. Load Submodel Outputs (from Kaggle Dataset) ===
print("\nLoading submodel outputs from Kaggle Dataset...")

submodel_files = {
    'vix': {
        'path': '../input/gold-prediction-submodels/vix.csv',
        'columns': ['vix_regime_probability', 'vix_mean_reversion_z', 'vix_persistence'],
        'date_col': 'date',
        'tz_aware': False,
    },
    'technical': {
        'path': '../input/gold-prediction-submodels/technical.csv',
        'columns': ['tech_trend_regime_prob', 'tech_mean_reversion_z', 'tech_volatility_regime'],
        'date_col': 'date',
        'tz_aware': True,  # CRITICAL: timezone-aware dates
    },
    'cross_asset': {
        'path': '../input/gold-prediction-submodels/cross_asset.csv',
        'columns': ['xasset_regime_prob', 'xasset_recession_signal', 'xasset_divergence'],
        'date_col': 'Date',
        'tz_aware': False,
    },
    'yield_curve': {
        'path': '../input/gold-prediction-submodels/yield_curve.csv',
        'columns': ['yc_spread_velocity_z', 'yc_curvature_z'],
        'date_col': 'index',  # Special: dates in first unnamed column
        'tz_aware': False,
    },
    'etf_flow': {
        'path': '../input/gold-prediction-submodels/etf_flow.csv',
        'columns': ['etf_regime_prob', 'etf_capital_intensity', 'etf_pv_divergence'],
        'date_col': 'Date',
        'tz_aware': False,
    },
    'inflation_expectation': {
        'path': '../input/gold-prediction-submodels/inflation_expectation.csv',
        'columns': ['ie_regime_prob', 'ie_anchoring_z', 'ie_gold_sensitivity_z'],
        'date_col': 'Unnamed: 0',  # Special: dates in unnamed column
        'tz_aware': False,
    },
    'options_market': {
        'path': '../input/gold-prediction-submodels/options_market.csv',
        'columns': ['options_risk_regime_prob'],
        'date_col': 'Date',
        'tz_aware': True,  # CRITICAL: timezone-aware dates (same as technical.csv)
    },
    'temporal_context': {
        'path': '../input/gold-prediction-submodels/temporal_context.csv',
        'columns': ['temporal_context_score'],
        'date_col': 'date',       # lowercase 'date'
        'tz_aware': False,         # no timezone in dates
    },
}

submodel_dfs = {}
for feature, spec in submodel_files.items():
    # Load CSV
    df = pd.read_csv(spec['path'])

    # Normalize date column
    date_col = spec['date_col']
    if spec['tz_aware']:
        # CRITICAL: timezone-aware dates require utc=True
        df['Date'] = pd.to_datetime(df[date_col], utc=True).dt.strftime('%Y-%m-%d')
    else:
        if date_col == 'index':
            # yield_curve.csv has dates in first unnamed column (index 0)
            df['Date'] = pd.to_datetime(df.iloc[:, 0]).dt.strftime('%Y-%m-%d')
        elif date_col == 'Unnamed: 0':
            # inflation_expectation.csv has dates in unnamed column
            df['Date'] = pd.to_datetime(df['Unnamed: 0']).dt.strftime('%Y-%m-%d')
        else:
            df['Date'] = pd.to_datetime(df[date_col]).dt.strftime('%Y-%m-%d')

    df = df[['Date'] + spec['columns']]
    df = df.set_index('Date')
    submodel_dfs[feature] = df
    print(f"  {feature}: {len(df)} rows")

print(f"\n✓ Data fetching complete")

## Feature Transformation and NaN Imputation

In [ ]:
# === Apply transformations (stationary conversion) ===
print("\nApplying transformations...")

# Create final feature DataFrame
final_df = base_features.copy()

# Base features (4 diff, 1 level)
final_df['real_rate_change'] = final_df['real_rate_real_rate'].diff()
final_df['dxy_change'] = final_df['dxy_dxy'].diff()
final_df['vix'] = final_df['vix_vix']  # Level (stationary)
final_df['yield_spread_change'] = final_df['yield_curve_yield_spread'].diff()
final_df['inflation_exp_change'] = final_df['inflation_expectation_inflation_expectation'].diff()

# Drop original raw columns
final_df = final_df.drop(columns=['real_rate_real_rate', 'dxy_dxy', 'vix_vix',
                                    'yield_curve_yield_spread', 'inflation_expectation_inflation_expectation'])

# === Merge submodel features ===
print("\nMerging submodel outputs...")
for feature, df in submodel_dfs.items():
    final_df = final_df.join(df, how='left')  # Left join preserves all base dates

print(f"  Features after merge: {final_df.shape[1]} columns, {len(final_df)} rows")

# === NaN Imputation (domain-specific) ===
print("\nApplying NaN imputation...")

nan_before = final_df.isna().sum().sum()
print(f"  NaN before imputation: {nan_before}")

# Regime probability columns → 0.5 (maximum uncertainty)
regime_cols = ['vix_regime_probability', 'tech_trend_regime_prob', 
               'xasset_regime_prob', 'etf_regime_prob', 'ie_regime_prob',
               'options_risk_regime_prob',
               'temporal_context_score',
               ]
for col in regime_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].fillna(0.5)

# Z-score columns → 0.0 (at mean)
z_cols = ['vix_mean_reversion_z', 'tech_mean_reversion_z', 
          'yc_spread_velocity_z', 'yc_curvature_z',
          'etf_capital_intensity', 'etf_pv_divergence',
          'ie_anchoring_z', 'ie_gold_sensitivity_z']
for col in z_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].fillna(0.0)

# Divergence/signal columns → 0.0 (neutral)
div_cols = ['xasset_recession_signal', 'xasset_divergence']
for col in div_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].fillna(0.0)

# Continuous state columns → median
cont_cols = ['tech_volatility_regime', 'vix_persistence']
for col in cont_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].fillna(final_df[col].median())

# Drop rows with NaN in target or base features (critical rows)
final_df = final_df.dropna(subset=['gold_return_next', 'real_rate_change', 'dxy_change', 
                                     'vix', 'yield_spread_change', 'inflation_exp_change'])

nan_after = final_df.isna().sum().sum()
print(f"  NaN after imputation: {nan_after}")
print(f"  Final dataset: {len(final_df)} rows")

# === Verify feature set ===
assert all(col in final_df.columns for col in FEATURE_COLUMNS), "Missing features after merge!"
assert TARGET in final_df.columns, "Target not found!"
print(f"\n✓ All {len(FEATURE_COLUMNS)} features present")
print(f"✓ Dataset shape: {final_df.shape}")
print(f"✓ Date range: {final_df.index.min()} to {final_df.index.max()}")

## Train/Val/Test Split (70/15/15)

In [ ]:
# === Train/Val/Test Split (70/15/15, time-series order) ===
n_total = len(final_df)
n_train = int(n_total * 0.70)
n_val = int(n_total * 0.15)

train_df = final_df.iloc[:n_train].copy()
val_df = final_df.iloc[n_train:n_train+n_val].copy()
test_df = final_df.iloc[n_train+n_val:].copy()

print(f"\n✓ Data split complete:")
print(f"  Train: {len(train_df)} rows ({len(train_df)/n_total*100:.1f}%) - {train_df.index.min()} to {train_df.index.max()}")
print(f"  Val:   {len(val_df)} rows ({len(val_df)/n_total*100:.1f}%) - {val_df.index.min()} to {val_df.index.max()}")
print(f"  Test:  {len(test_df)} rows ({len(test_df)/n_total*100:.1f}%) - {test_df.index.min()} to {test_df.index.max()}")
print(f"  Total: {n_total} rows")
print(f"  Samples per feature: {n_train / len(FEATURE_COLUMNS):.1f}:1 (train)")

# Verify no data leakage
assert train_df.index.max() < val_df.index.min(), "Train-val overlap detected!"
assert val_df.index.max() < test_df.index.min(), "Val-test overlap detected!"
print(f"\n✓ No time-series leakage detected")
print("="*60)

# Prepare arrays for training
X_train = train_df[FEATURE_COLUMNS].values
y_train = train_df[TARGET].values

X_val = val_df[FEATURE_COLUMNS].values
y_val = val_df[TARGET].values

X_test = test_df[FEATURE_COLUMNS].values
y_test = test_df[TARGET].values

# Store dates for output
dates_train = train_df.index
dates_val = val_df.index
dates_test = test_df.index

print(f"\nArray shapes:")
print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"  X_val:   {X_val.shape}, y_val:   {y_val.shape}")
print(f"  X_test:  {X_test.shape}, y_test:  {y_test.shape}")

## Metric Functions

In [ ]:
def compute_direction_accuracy(y_true, y_pred):
    """Direction accuracy, excluding zeros."""
    mask = (y_true != 0) & (y_pred != 0)
    if mask.sum() == 0:
        return 0.0
    return (np.sign(y_pred[mask]) == np.sign(y_true[mask])).mean()

def compute_mae(y_true, y_pred):
    """Mean Absolute Error."""
    return np.abs(y_pred - y_true).mean()

def compute_sharpe_trade_cost(y_true, y_pred, cost_bps=5.0):
    """Sharpe ratio with position-change cost (5bps per change)."""
    positions = np.sign(y_pred)
    
    # Strategy returns (position * actual return)
    strategy_returns = positions * y_true / 100.0  # Convert % to decimal
    
    # Position changes
    position_changes = np.abs(np.diff(positions, prepend=0))
    trade_costs = position_changes * (cost_bps / 10000.0)  # 5bps = 0.0005
    
    # Net returns
    net_returns = strategy_returns - trade_costs
    
    # Annualized Sharpe (252 trading days)
    if len(net_returns) < 2 or net_returns.std() == 0:
        return 0.0
    return (net_returns.mean() / net_returns.std()) * np.sqrt(252)

def compute_hcda(y_true, y_pred, threshold_percentile=80):
    """High-confidence direction accuracy (top 20% by |prediction|)."""
    threshold = np.percentile(np.abs(y_pred), threshold_percentile)
    hc_mask = np.abs(y_pred) > threshold
    
    if hc_mask.sum() == 0:
        return 0.0, 0.0
    
    coverage = hc_mask.sum() / len(y_pred)
    hc_pred = y_pred[hc_mask]
    hc_actual = y_true[hc_mask]
    
    mask = (hc_actual != 0) & (hc_pred != 0)
    if mask.sum() == 0:
        return 0.0, coverage
    
    da = (np.sign(hc_pred[mask]) == np.sign(hc_actual[mask])).mean()
    return da, coverage

def compute_hcda_bootstrap(y_true, y_pred, bootstrap_std, threshold_percentile=80):
    """
    HCDA using bootstrap variance-based confidence.
    High confidence = LOW variance (certain predictions)
    Top 20% by inverse variance: 1 / (1 + std)
    """
    confidence = 1.0 / (1.0 + bootstrap_std)  # Higher confidence when std is low
    threshold = np.percentile(confidence, threshold_percentile)
    hc_mask = confidence > threshold
    
    if hc_mask.sum() == 0:
        return 0.0, 0.0
    
    coverage = hc_mask.sum() / len(y_pred)
    hc_pred = y_pred[hc_mask]
    hc_actual = y_true[hc_mask]
    
    mask = (hc_actual != 0) & (hc_pred != 0)
    if mask.sum() == 0:
        return 0.0, coverage
    
    da = (np.sign(hc_pred[mask]) == np.sign(hc_actual[mask])).mean()
    return da, coverage

def compute_composite_objective(y_true, y_pred, train_da=None, val_da=None):
    """Compute composite objective (same formula as XGBoost Optuna)."""
    da = compute_direction_accuracy(y_true, y_pred)
    mae = compute_mae(y_true, y_pred)
    sharpe = compute_sharpe_trade_cost(y_true, y_pred)
    hc_da, _ = compute_hcda(y_true, y_pred, threshold_percentile=80)
    
    # Normalize metrics to [0, 1]
    sharpe_norm = np.clip((sharpe + 3.0) / 6.0, 0.0, 1.0)   # [-3, +3] -> [0, 1]
    da_norm = np.clip((da * 100 - 40.0) / 30.0, 0.0, 1.0)   # [40%, 70%] -> [0, 1]
    mae_norm = np.clip((1.0 - mae) / 0.5, 0.0, 1.0)         # [0.5%, 1.0%] -> [0, 1]
    hc_da_norm = np.clip((hc_da * 100 - 40.0) / 30.0, 0.0, 1.0)  # [40%, 70%] -> [0, 1]
    
    # Overfitting penalty (if train_da and val_da provided)
    overfit_penalty = 0.0
    if train_da is not None and val_da is not None:
        da_gap = (train_da - val_da) * 100  # In percentage points
        overfit_penalty = max(0.0, (da_gap - 10.0) / 30.0)  # 0 if gap<=10pp, up to 1.0 if gap=40pp
    
    # Weighted composite
    objective = (
        0.40 * sharpe_norm +
        0.30 * da_norm +
        0.10 * mae_norm +
        0.20 * hc_da_norm
    ) - 0.30 * overfit_penalty
    
    return objective

print("Metric functions defined")

## STEP 1: XGBoost HPO (100 trials) - Same as Attempt 7

In [ ]:
def xgboost_objective(trial):
    """XGBoost Optuna objective (same as Attempt 7)."""
    
    # === Sample hyperparameters (ATTEMPT 6/7 RANGES) ===
    params = {
        'objective': 'reg:squarederror',
        'max_depth': trial.suggest_int('max_depth', 2, 4),
        'min_child_weight': trial.suggest_int('min_child_weight', 12, 25),
        'subsample': trial.suggest_float('subsample', 0.4, 0.85),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 0.7),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 15.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.5, 10.0, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.05, log=True),
        'tree_method': 'hist',
        'eval_metric': 'rmse',
        'verbosity': 0,
        'seed': 42 + trial.number,
    }
    
    n_estimators = trial.suggest_int('n_estimators', 100, 800)
    
    # === Train model ===
    model = xgb.XGBRegressor(**params, n_estimators=n_estimators, early_stopping_rounds=100)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    # === Predictions ===
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    
    # === Compute composite objective ===
    train_da = compute_direction_accuracy(y_train, train_pred)
    val_da = compute_direction_accuracy(y_val, val_pred)
    objective = compute_composite_objective(y_val, val_pred, train_da=train_da, val_da=val_da)
    
    # === Log trial details ===
    val_mae = compute_mae(y_val, val_pred)
    val_sharpe = compute_sharpe_trade_cost(y_val, val_pred)
    val_hc_da, val_hc_coverage = compute_hcda(y_val, val_pred, threshold_percentile=80)
    da_gap = (train_da - val_da) * 100
    
    trial.set_user_attr('val_da', float(val_da))
    trial.set_user_attr('val_mae', float(val_mae))
    trial.set_user_attr('val_sharpe', float(val_sharpe))
    trial.set_user_attr('val_hc_da', float(val_hc_da))
    trial.set_user_attr('val_hc_coverage', float(val_hc_coverage))
    trial.set_user_attr('train_da', float(train_da))
    trial.set_user_attr('da_gap_pp', float(da_gap))
    trial.set_user_attr('n_estimators_used',
                         int(model.best_iteration + 1) if hasattr(model, 'best_iteration')
                         and model.best_iteration is not None else n_estimators)
    
    return objective

print("XGBoost Optuna objective function defined (same as Attempt 7)")

In [ ]:
print("\n" + "="*60)
print("RUNNING XGBOOST OPTUNA HPO (100 trials, 2-hour timeout)")
print("="*60)

xgb_study = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42),
)

xgb_study.optimize(
    xgboost_objective,
    n_trials=100,
    timeout=7200,  # 2 hours
    show_progress_bar=True
)

print(f"\nXGBoost optimization complete")
print(f"  Trials completed: {len(xgb_study.trials)}")
print(f"  Best value: {xgb_study.best_value:.4f}")
print(f"\nBest hyperparameters:")
for k, v in xgb_study.best_params.items():
    print(f"  {k}: {v}")

best_xgb_trial = xgb_study.best_trial
print(f"\nBest XGBoost trial validation metrics:")
print(f"  DA:     {best_xgb_trial.user_attrs['val_da']*100:.2f}%")
print(f"  HCDA:   {best_xgb_trial.user_attrs['val_hc_da']*100:.2f}%")
print(f"  MAE:    {best_xgb_trial.user_attrs['val_mae']:.4f}%")
print(f"  Sharpe: {best_xgb_trial.user_attrs['val_sharpe']:.2f}")
print(f"  DA gap: {best_xgb_trial.user_attrs['da_gap_pp']:.2f}pp")

## STEP 2: Ridge Regression HPO (20 trials)

In [ ]:
print("\n" + "="*60)
print("PREPARING DATA FOR RIDGE REGRESSION (StandardScaler)")
print("="*60)

# Ridge needs scaled features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"\n✓ StandardScaler fitted on training data")
print(f"  Scaled train: mean={X_train_scaled.mean():.4f}, std={X_train_scaled.std():.4f}")
print(f"  Scaled val:   mean={X_val_scaled.mean():.4f}, std={X_val_scaled.std():.4f}")

In [ ]:
def ridge_objective(trial):
    """Ridge Optuna objective."""
    
    # === Sample alpha ===
    alpha = trial.suggest_float('ridge_alpha', 0.01, 100.0, log=True)
    
    # === Train Ridge ===
    ridge = Ridge(alpha=alpha, random_state=42)
    ridge.fit(X_train_scaled, y_train)
    
    # === Predictions ===
    train_pred = ridge.predict(X_train_scaled)
    val_pred = ridge.predict(X_val_scaled)
    
    # === Compute composite objective (same formula as XGBoost) ===
    train_da = compute_direction_accuracy(y_train, train_pred)
    val_da = compute_direction_accuracy(y_val, val_pred)
    objective = compute_composite_objective(y_val, val_pred, train_da=train_da, val_da=val_da)
    
    # === Log trial details ===
    val_mae = compute_mae(y_val, val_pred)
    val_sharpe = compute_sharpe_trade_cost(y_val, val_pred)
    val_hc_da, val_hc_coverage = compute_hcda(y_val, val_pred, threshold_percentile=80)
    da_gap = (train_da - val_da) * 100
    
    trial.set_user_attr('val_da', float(val_da))
    trial.set_user_attr('val_mae', float(val_mae))
    trial.set_user_attr('val_sharpe', float(val_sharpe))
    trial.set_user_attr('val_hc_da', float(val_hc_da))
    trial.set_user_attr('val_hc_coverage', float(val_hc_coverage))
    trial.set_user_attr('train_da', float(train_da))
    trial.set_user_attr('da_gap_pp', float(da_gap))
    
    return objective

print("Ridge Optuna objective function defined")

In [ ]:
print("\n" + "="*60)
print("RUNNING RIDGE OPTUNA HPO (20 trials, 10-minute timeout)")
print("="*60)

ridge_study = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=43),  # Different seed from XGBoost
)

ridge_study.optimize(
    ridge_objective,
    n_trials=20,
    timeout=600,  # 10 minutes (Ridge is fast)
    show_progress_bar=True
)

print(f"\nRidge optimization complete")
print(f"  Trials completed: {len(ridge_study.trials)}")
print(f"  Best value: {ridge_study.best_value:.4f}")
print(f"  Best alpha: {ridge_study.best_params['ridge_alpha']:.4f}")

best_ridge_trial = ridge_study.best_trial
print(f"\nBest Ridge trial validation metrics:")
print(f"  DA:     {best_ridge_trial.user_attrs['val_da']*100:.2f}%")
print(f"  HCDA:   {best_ridge_trial.user_attrs['val_hc_da']*100:.2f}%")
print(f"  MAE:    {best_ridge_trial.user_attrs['val_mae']:.4f}%")
print(f"  Sharpe: {best_ridge_trial.user_attrs['val_sharpe']:.2f}")
print(f"  DA gap: {best_ridge_trial.user_attrs['da_gap_pp']:.2f}pp")

## STEP 3: Train Final XGBoost and Ridge Models

In [ ]:
print("\n" + "="*60)
print("TRAINING FINAL XGBOOST MODEL")
print("="*60)

# Train final XGBoost with best params
xgb_best_params = xgb_study.best_params
final_xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    max_depth=xgb_best_params['max_depth'],
    min_child_weight=xgb_best_params['min_child_weight'],
    subsample=xgb_best_params['subsample'],
    colsample_bytree=xgb_best_params['colsample_bytree'],
    reg_lambda=xgb_best_params['reg_lambda'],
    reg_alpha=xgb_best_params['reg_alpha'],
    learning_rate=xgb_best_params['learning_rate'],
    tree_method='hist',
    eval_metric='rmse',
    verbosity=0,
    seed=42,
    n_estimators=xgb_best_params['n_estimators'],
    early_stopping_rounds=100
)

final_xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

xgb_pred_train = final_xgb_model.predict(X_train)
xgb_pred_val = final_xgb_model.predict(X_val)
xgb_pred_test = final_xgb_model.predict(X_test)

print("✓ XGBoost model trained")
print(f"  Train predictions: mean={xgb_pred_train.mean():.4f}, std={xgb_pred_train.std():.4f}")
print(f"  Val predictions:   mean={xgb_pred_val.mean():.4f}, std={xgb_pred_val.std():.4f}")
print(f"  Test predictions:  mean={xgb_pred_test.mean():.4f}, std={xgb_pred_test.std():.4f}")

In [ ]:
print("\n" + "="*60)
print("TRAINING FINAL RIDGE MODEL")
print("="*60)

# Train final Ridge with best alpha
ridge_best_alpha = ridge_study.best_params['ridge_alpha']
final_ridge_model = Ridge(alpha=ridge_best_alpha, random_state=42)
final_ridge_model.fit(X_train_scaled, y_train)

ridge_pred_train = final_ridge_model.predict(X_train_scaled)
ridge_pred_val = final_ridge_model.predict(X_val_scaled)
ridge_pred_test = final_ridge_model.predict(X_test_scaled)

print("✓ Ridge model trained")
print(f"  Train predictions: mean={ridge_pred_train.mean():.4f}, std={ridge_pred_train.std():.4f}")
print(f"  Val predictions:   mean={ridge_pred_val.mean():.4f}, std={ridge_pred_val.std():.4f}")
print(f"  Test predictions:  mean={ridge_pred_test.mean():.4f}, std={ridge_pred_test.std():.4f}")

## STEP 4: Optimize Ensemble Weight on Validation Set

In [ ]:
print("\n" + "="*60)
print("OPTIMIZING ENSEMBLE WEIGHT ON VALIDATION SET")
print("="*60)

# Grid search for optimal weight: w * xgb + (1-w) * ridge
best_w = 0.5
best_composite = -1.0
weight_results = []

for w in np.arange(0.0, 1.05, 0.05):
    # Ensemble prediction
    ensemble_pred = w * xgb_pred_val + (1.0 - w) * ridge_pred_val
    
    # Compute composite objective (no overfitting penalty for weight selection)
    composite = compute_composite_objective(y_val, ensemble_pred)
    
    # Compute individual metrics
    da = compute_direction_accuracy(y_val, ensemble_pred)
    mae = compute_mae(y_val, ensemble_pred)
    sharpe = compute_sharpe_trade_cost(y_val, ensemble_pred)
    hcda, _ = compute_hcda(y_val, ensemble_pred)
    
    weight_results.append({
        'weight': w,
        'composite': composite,
        'da': da,
        'mae': mae,
        'sharpe': sharpe,
        'hcda': hcda,
    })
    
    if composite > best_composite:
        best_composite = composite
        best_w = w

print(f"\n✓ Optimal ensemble weight found: w = {best_w:.2f}")
print(f"  Composite objective: {best_composite:.4f}")
print(f"\nInterpretation:")
print(f"  XGBoost contribution: {best_w*100:.0f}%")
print(f"  Ridge contribution:   {(1-best_w)*100:.0f}%")

# Print weight search results (top 5)
print(f"\nTop 5 weights by composite objective (validation):")
for result in sorted(weight_results, key=lambda x: x['composite'], reverse=True)[:5]:
    print(f"  w={result['weight']:.2f}: composite={result['composite']:.4f}, "
          f"DA={result['da']*100:.2f}%, Sharpe={result['sharpe']:.2f}, "
          f"MAE={result['mae']:.4f}%, HCDA={result['hcda']*100:.2f}%")

## STEP 5: Generate Final Ensemble Predictions

In [ ]:
print("\n" + "="*60)
print(f"GENERATING FINAL ENSEMBLE PREDICTIONS (w={best_w:.2f})")
print("="*60)

# Ensemble predictions for all splits
pred_train = best_w * xgb_pred_train + (1.0 - best_w) * ridge_pred_train
pred_val = best_w * xgb_pred_val + (1.0 - best_w) * ridge_pred_val
pred_test = best_w * xgb_pred_test + (1.0 - best_w) * ridge_pred_test

# Combine all predictions for full dataset
pred_full = np.concatenate([pred_train, pred_val, pred_test])
dates_full = pd.Index(list(dates_train) + list(dates_val) + list(dates_test))
y_full = np.concatenate([y_train, y_val, y_test])

print("\nEnsemble predictions generated:")
print(f"  Train: mean={pred_train.mean():.4f}, std={pred_train.std():.4f}")
print(f"  Val:   mean={pred_val.mean():.4f}, std={pred_val.std():.4f}")
print(f"  Test:  mean={pred_test.mean():.4f}, std={pred_test.std():.4f}")

## POST-TRAINING STEP 1: OLS Output Scaling

In [ ]:
print("\n" + "="*60)
print("OLS OUTPUT SCALING")
print("="*60)

# Compute optimal scaling factor from validation set
numerator = np.sum(pred_val * y_val)
denominator = np.sum(pred_val ** 2)
alpha_ols = numerator / denominator if denominator != 0 else 1.0
alpha_ols = np.clip(alpha_ols, 0.5, 10.0)  # Cap at [0.5, 10.0]

print(f"\nOLS scaling factor: {alpha_ols:.2f}")

# Apply scaling to all predictions (for MAE computation only)
scaled_pred_train = pred_train * alpha_ols
scaled_pred_val = pred_val * alpha_ols
scaled_pred_test = pred_test * alpha_ols
scaled_pred_full = pred_full * alpha_ols

# MAE comparison
mae_raw = np.mean(np.abs(pred_test - y_test))
mae_scaled = np.mean(np.abs(scaled_pred_test - y_test))
print(f"\nMAE (raw):    {mae_raw:.4f}%")
print(f"MAE (scaled): {mae_scaled:.4f}%")
print(f"MAE delta:    {mae_scaled - mae_raw:+.4f}%")

# Verify: DA and Sharpe unchanged by scaling
da_raw = compute_direction_accuracy(y_test, pred_test)
da_scaled = compute_direction_accuracy(y_test, scaled_pred_test)
assert abs(da_raw - da_scaled) < 1e-10, "Scaling changed DA!"
print("\n✓ DA and Sharpe: unchanged by scaling (verified)")

# Select better MAE for reporting
use_scaled = mae_scaled < mae_raw
if use_scaled:
    print(f"\n✓ Using SCALED predictions for MAE (improvement: {mae_raw - mae_scaled:.4f}%)")
else:
    print(f"\n✓ Using RAW predictions for MAE (scaling degraded by {mae_scaled - mae_raw:.4f}%)")

## POST-TRAINING STEP 2: Bootstrap Ensemble for Confidence

In [ ]:
print("\n" + "="*60)
print("BOOTSTRAP ENSEMBLE CONFIDENCE SCORING")
print("="*60)

# === Train 5 XGBoost models with different random seeds ===
print("\nTraining XGBoost bootstrap ensemble (5 models)...")
bootstrap_models = []
bootstrap_seeds = [42, 43, 44, 45, 46]

for i, seed in enumerate(bootstrap_seeds):
    print(f"  Training XGBoost model {i+1}/5 (seed={seed})...")
    
    # Create model with new seed
    model_boot = xgb.XGBRegressor(
        objective='reg:squarederror',
        max_depth=xgb_best_params['max_depth'],
        min_child_weight=xgb_best_params['min_child_weight'],
        subsample=xgb_best_params['subsample'],
        colsample_bytree=xgb_best_params['colsample_bytree'],
        reg_lambda=xgb_best_params['reg_lambda'],
        reg_alpha=xgb_best_params['reg_alpha'],
        learning_rate=xgb_best_params['learning_rate'],
        tree_method='hist',
        eval_metric='rmse',
        verbosity=0,
        seed=seed,  # Different seed for each model
        n_estimators=xgb_best_params['n_estimators'],
        early_stopping_rounds=100
    )
    
    model_boot.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    bootstrap_models.append(model_boot)

print(f"\n✓ XGBoost bootstrap ensemble trained: {len(bootstrap_models)} models")

# === Generate XGBoost predictions from all 5 models ===
print("\nGenerating predictions from XGBoost ensemble...")
xgb_ensemble_preds_train = np.array([m.predict(X_train) for m in bootstrap_models])
xgb_ensemble_preds_val = np.array([m.predict(X_val) for m in bootstrap_models])
xgb_ensemble_preds_test = np.array([m.predict(X_test) for m in bootstrap_models])

# === Ridge prediction (deterministic, single model) ===
print("\nRidge predictions (single model, deterministic)...")
# Ridge already computed: ridge_pred_train, ridge_pred_val, ridge_pred_test

# === Compute ensemble predictions for each bootstrap XGBoost model ===
ensemble_preds_train = best_w * xgb_ensemble_preds_train + (1.0 - best_w) * ridge_pred_train
ensemble_preds_val = best_w * xgb_ensemble_preds_val + (1.0 - best_w) * ridge_pred_val
ensemble_preds_test = best_w * xgb_ensemble_preds_test + (1.0 - best_w) * ridge_pred_test

# === Compute variance across ensemble (only XGBoost varies, Ridge is constant) ===
bootstrap_std_train = np.std(ensemble_preds_train, axis=0)
bootstrap_std_val = np.std(ensemble_preds_val, axis=0)
bootstrap_std_test = np.std(ensemble_preds_test, axis=0)

# Bootstrap confidence = 1 / (1 + std)
bootstrap_conf_train = 1.0 / (1.0 + bootstrap_std_train)
bootstrap_conf_val = 1.0 / (1.0 + bootstrap_std_val)
bootstrap_conf_test = 1.0 / (1.0 + bootstrap_std_test)

print(f"\nBootstrap variance statistics (test set):")
print(f"  Std range: [{bootstrap_std_test.min():.4f}, {bootstrap_std_test.max():.4f}]")
print(f"  Std mean:  {bootstrap_std_test.mean():.4f}")
print(f"  Confidence range: [{bootstrap_conf_test.min():.4f}, {bootstrap_conf_test.max():.4f}]")
print(f"  Confidence mean:  {bootstrap_conf_test.mean():.4f}")

# Compute HCDA with BOTH methods (using ensemble predictions)
hcda_bootstrap_test, hcda_bootstrap_cov = compute_hcda_bootstrap(y_test, pred_test, bootstrap_std_test)
hcda_pred_test, hcda_pred_cov = compute_hcda(y_test, pred_test)  # Original |prediction| method

print(f"\nHCDA comparison (test set):")
print(f"  Bootstrap variance: {hcda_bootstrap_test*100:.2f}% (N={int(hcda_bootstrap_cov*len(y_test))})")
print(f"  |prediction|:       {hcda_pred_test*100:.2f}% (N={int(hcda_pred_cov*len(y_test))})")
print(f"  Improvement:        {(hcda_bootstrap_test - hcda_pred_test)*100:+.2f}pp")

# Select better HCDA method
use_bootstrap_hcda = hcda_bootstrap_test > hcda_pred_test
if use_bootstrap_hcda:
    print(f"\n✓ Using bootstrap variance for HCDA (better by {(hcda_bootstrap_test - hcda_pred_test)*100:.2f}pp)")
    primary_hcda_method = 'bootstrap'
    primary_hcda_value = hcda_bootstrap_test
else:
    print(f"\n✓ Using |prediction| for HCDA (better by {(hcda_pred_test - hcda_bootstrap_test)*100:.2f}pp)")
    primary_hcda_method = 'pred'
    primary_hcda_value = hcda_pred_test

## Evaluation on All Splits

In [ ]:
print("\n" + "="*60)
print("FINAL EVALUATION")
print("="*60)

# Compute metrics for all splits (using ensemble predictions)
metrics_all = {}
for split_name, y_true, y_pred_raw, y_pred_scaled in [
    ('train', y_train, pred_train, scaled_pred_train),
    ('val', y_val, pred_val, scaled_pred_val),
    ('test', y_test, pred_test, scaled_pred_test),
]:
    da = compute_direction_accuracy(y_true, y_pred_raw)  # DA uses raw predictions
    mae_raw_split = compute_mae(y_true, y_pred_raw)
    mae_scaled_split = compute_mae(y_true, y_pred_scaled)
    mae = min(mae_raw_split, mae_scaled_split)  # Report better MAE
    sharpe = compute_sharpe_trade_cost(y_true, y_pred_raw)  # Sharpe uses raw predictions
    hc_da, hc_coverage = compute_hcda(y_true, y_pred_raw, threshold_percentile=80)
    
    metrics_all[split_name] = {
        'direction_accuracy': float(da),
        'high_confidence_da': float(hc_da),
        'high_confidence_coverage': float(hc_coverage),
        'mae': float(mae),
        'mae_raw': float(mae_raw_split),
        'mae_scaled': float(mae_scaled_split),
        'sharpe_ratio': float(sharpe),
    }

# Print metrics
for split_name in ['train', 'val', 'test']:
    m = metrics_all[split_name]
    print(f"\n{split_name.upper()}:")
    print(f"  DA:     {m['direction_accuracy']*100:.2f}%")
    print(f"  HCDA:   {m['high_confidence_da']*100:.2f}% (coverage: {m['high_confidence_coverage']*100:.1f}%)")
    print(f"  MAE:    {m['mae']:.4f}% (raw: {m['mae_raw']:.4f}%, scaled: {m['mae_scaled']:.4f}%)")
    print(f"  Sharpe: {m['sharpe_ratio']:.2f}")

# Overfitting analysis
train_test_da_gap = (metrics_all['train']['direction_accuracy'] - metrics_all['test']['direction_accuracy']) * 100
print(f"\nOVERFITTING:")
print(f"  Train-Test DA gap: {train_test_da_gap:.2f}pp (target: <10pp)")

# Target evaluation (using BEST of bootstrap/|pred| HCDA)
test_m = metrics_all['test']
targets_met = [
    test_m['direction_accuracy'] > 0.56,
    primary_hcda_value > 0.60,  # Use better of bootstrap/|pred|
    test_m['mae'] < 0.0075,
    test_m['sharpe_ratio'] > 0.8,
]

print(f"\nTARGET STATUS:")
print(f"  DA > 56%:     {'✓' if targets_met[0] else '✗'} ({test_m['direction_accuracy']*100:.2f}%)")
print(f"  HCDA > 60%:   {'✓' if targets_met[1] else '✗'} ({primary_hcda_value*100:.2f}% via {primary_hcda_method})")
print(f"  MAE < 0.75%:  {'✓' if targets_met[2] else '✗'} ({test_m['mae']:.4f}%)")
print(f"  Sharpe > 0.8: {'✓' if targets_met[3] else '✗'} ({test_m['sharpe_ratio']:.2f})")
print(f"\nTargets passed: {sum(targets_met)}/4")

## Model Comparison: Ensemble vs Individual Models

In [ ]:
print("\n" + "="*60)
print("MODEL COMPARISON (test set)")
print("="*60)

# Individual model metrics
xgb_only_da = compute_direction_accuracy(y_test, xgb_pred_test)
xgb_only_mae = compute_mae(y_test, xgb_pred_test)
xgb_only_sharpe = compute_sharpe_trade_cost(y_test, xgb_pred_test)
xgb_only_hcda, _ = compute_hcda(y_test, xgb_pred_test)

ridge_only_da = compute_direction_accuracy(y_test, ridge_pred_test)
ridge_only_mae = compute_mae(y_test, ridge_pred_test)
ridge_only_sharpe = compute_sharpe_trade_cost(y_test, ridge_pred_test)
ridge_only_hcda, _ = compute_hcda(y_test, ridge_pred_test)

# Ensemble metrics
ensemble_da = test_m['direction_accuracy']
ensemble_mae = test_m['mae']
ensemble_sharpe = test_m['sharpe_ratio']
ensemble_hcda = primary_hcda_value

print(f"\nXGBoost Only (w=1.0):")
print(f"  DA:     {xgb_only_da*100:.2f}%")
print(f"  HCDA:   {xgb_only_hcda*100:.2f}%")
print(f"  MAE:    {xgb_only_mae:.4f}%")
print(f"  Sharpe: {xgb_only_sharpe:.2f}")

print(f"\nRidge Only (w=0.0):")
print(f"  DA:     {ridge_only_da*100:.2f}%")
print(f"  HCDA:   {ridge_only_hcda*100:.2f}%")
print(f"  MAE:    {ridge_only_mae:.4f}%")
print(f"  Sharpe: {ridge_only_sharpe:.2f}")

print(f"\nEnsemble (w={best_w:.2f}):")
print(f"  DA:     {ensemble_da*100:.2f}%")
print(f"  HCDA:   {ensemble_hcda*100:.2f}%")
print(f"  MAE:    {ensemble_mae:.4f}%")
print(f"  Sharpe: {ensemble_sharpe:.2f}")

print(f"\nEnsemble vs XGBoost Only:")
print(f"  DA delta:     {(ensemble_da - xgb_only_da)*100:+.2f}pp")
print(f"  HCDA delta:   {(ensemble_hcda - xgb_only_hcda)*100:+.2f}pp")
print(f"  MAE delta:    {(ensemble_mae - xgb_only_mae):+.4f}%")
print(f"  Sharpe delta: {(ensemble_sharpe - xgb_only_sharpe):+.2f}")

print(f"\nEnsemble vs Ridge Only:")
print(f"  DA delta:     {(ensemble_da - ridge_only_da)*100:+.2f}pp")
print(f"  HCDA delta:   {(ensemble_hcda - ridge_only_hcda)*100:+.2f}pp")
print(f"  MAE delta:    {(ensemble_mae - ridge_only_mae):+.4f}%")
print(f"  Sharpe delta: {(ensemble_sharpe - ridge_only_sharpe):+.2f}")

## Save Results

In [ ]:
print("\n" + "="*60)
print("SAVING RESULTS")
print("="*60)

# 1. predictions.csv (full dataset, with BOTH raw and scaled)
split_labels = ['train'] * len(dates_train) + ['val'] * len(dates_val) + ['test'] * len(dates_test)
predictions_df = pd.DataFrame({
    'date': dates_full,
    'split': split_labels,
    'actual': y_full,
    'prediction_raw': pred_full,
    'prediction_scaled': scaled_pred_full,
    'direction_correct': (np.sign(pred_full) == np.sign(y_full)).astype(int),
    'abs_prediction': np.abs(pred_full),
})

# Add high_confidence flags (80th percentile for both methods)
threshold_80_pred = np.percentile(np.abs(pred_full), 80)
predictions_df['high_confidence_pred'] = (predictions_df['abs_prediction'] > threshold_80_pred).astype(int)

# Bootstrap confidence for full dataset
bootstrap_conf_full = np.concatenate([bootstrap_conf_train, bootstrap_conf_val, bootstrap_conf_test])
threshold_80_bootstrap = np.percentile(bootstrap_conf_full, 80)
predictions_df['bootstrap_confidence'] = bootstrap_conf_full
predictions_df['high_confidence_bootstrap'] = (predictions_df['bootstrap_confidence'] > threshold_80_bootstrap).astype(int)

# Bootstrap std for full dataset
bootstrap_std_full = np.concatenate([bootstrap_std_train, bootstrap_std_val, bootstrap_std_test])
predictions_df['bootstrap_std'] = bootstrap_std_full

predictions_df.to_csv('predictions.csv', index=False)
print("✓ Saved predictions.csv")

# 2. test_predictions.csv (test set only)
test_predictions_df = predictions_df[predictions_df['split'] == 'test'].copy()
test_predictions_df.to_csv('test_predictions.csv', index=False)
print("✓ Saved test_predictions.csv")

# 3. submodel_output.csv (for pipeline compatibility)
predictions_df.to_csv('submodel_output.csv', index=False)
print("✓ Saved submodel_output.csv")

# 4. model.json (XGBoost model)
final_xgb_model.save_model('model.json')
print("✓ Saved model.json (XGBoost)")

# 5. training_result.json
training_result = {
    'feature': 'meta_model',
    'attempt': 15,
    'timestamp': datetime.now().isoformat(),
    'architecture': 'Ridge + XGBoost Weighted Average Ensemble',
    'phase': '3_meta_model',
    
    'model_config': {
        'n_features': 24,
        'train_samples': len(X_train),
        'val_samples': len(X_val),
        'test_samples': len(X_test),
        'samples_per_feature_ratio': len(X_train) / 24,
        'ensemble_weight': float(best_w),
        'ensemble_interpretation': f"XGBoost {best_w*100:.0f}% + Ridge {(1-best_w)*100:.0f}%",
    },
    
    'xgboost_search': {
        'n_trials': len(xgb_study.trials),
        'best_value': float(xgb_study.best_value),
        'best_params': xgb_best_params,
    },
    
    'ridge_search': {
        'n_trials': len(ridge_study.trials),
        'best_value': float(ridge_study.best_value),
        'best_alpha': float(ridge_best_alpha),
    },
    
    'ensemble_optimization': {
        'optimal_weight': float(best_w),
        'optimal_composite': float(best_composite),
        'weight_search_results': weight_results,
    },
    
    'bootstrap_analysis': {
        'bootstrap_ensemble_size': 5,
        'bootstrap_seeds': bootstrap_seeds,
        'bootstrap_std_range_test': [float(bootstrap_std_test.min()), float(bootstrap_std_test.max())],
        'bootstrap_std_mean_test': float(bootstrap_std_test.mean()),
        'bootstrap_conf_range_test': [float(bootstrap_conf_test.min()), float(bootstrap_conf_test.max())],
        'bootstrap_conf_mean_test': float(bootstrap_conf_test.mean()),
        'hcda_bootstrap': float(hcda_bootstrap_test),
        'hcda_pred': float(hcda_pred_test),
        'hcda_improvement': float(hcda_bootstrap_test - hcda_pred_test),
    },
    
    'ols_scaling': {
        'alpha_ols': float(alpha_ols),
        'mae_raw': float(mae_raw),
        'mae_scaled': float(mae_scaled),
        'mae_improvement': float(mae_raw - mae_scaled),
    },
    
    'primary_hcda_method': primary_hcda_method,
    'primary_hcda_value': float(primary_hcda_value),
    'primary_mae': float(min(mae_raw, mae_scaled)),
    
    'metrics': metrics_all,
    
    'individual_model_comparison': {
        'xgb_only': {
            'da': float(xgb_only_da),
            'hcda': float(xgb_only_hcda),
            'mae': float(xgb_only_mae),
            'sharpe': float(xgb_only_sharpe),
        },
        'ridge_only': {
            'da': float(ridge_only_da),
            'hcda': float(ridge_only_hcda),
            'mae': float(ridge_only_mae),
            'sharpe': float(ridge_only_sharpe),
        },
        'ensemble': {
            'da': float(ensemble_da),
            'hcda': float(ensemble_hcda),
            'mae': float(ensemble_mae),
            'sharpe': float(ensemble_sharpe),
        },
        'ensemble_vs_xgb': {
            'da_delta_pp': float((ensemble_da - xgb_only_da) * 100),
            'hcda_delta_pp': float((ensemble_hcda - xgb_only_hcda) * 100),
            'mae_delta': float(ensemble_mae - xgb_only_mae),
            'sharpe_delta': float(ensemble_sharpe - xgb_only_sharpe),
        },
        'ensemble_vs_ridge': {
            'da_delta_pp': float((ensemble_da - ridge_only_da) * 100),
            'hcda_delta_pp': float((ensemble_hcda - ridge_only_hcda) * 100),
            'mae_delta': float(ensemble_mae - ridge_only_mae),
            'sharpe_delta': float(ensemble_sharpe - ridge_only_sharpe),
        },
    },
    
    'target_evaluation': {
        'direction_accuracy': {
            'target': '> 56.0%',
            'actual': f"{test_m['direction_accuracy']*100:.2f}%",
            'gap': f"{(test_m['direction_accuracy'] - 0.56)*100:+.2f}pp",
            'passed': bool(targets_met[0]),
        },
        'high_confidence_da': {
            'target': '> 60.0%',
            'actual': f"{primary_hcda_value*100:.2f}%",
            'gap': f"{(primary_hcda_value - 0.60)*100:+.2f}pp",
            'passed': bool(targets_met[1]),
            'method_used': primary_hcda_method,
        },
        'mae': {
            'target': '< 0.75%',
            'actual': f"{test_m['mae']:.4f}%",
            'gap': f"{(0.0075 - test_m['mae']):.4f}%",
            'passed': bool(targets_met[2]),
        },
        'sharpe_ratio': {
            'target': '> 0.80',
            'actual': f"{test_m['sharpe_ratio']:.2f}",
            'gap': f"{(test_m['sharpe_ratio'] - 0.8):+.2f}",
            'passed': bool(targets_met[3]),
        },
    },
    
    'targets_passed': sum(targets_met),
    'targets_total': 4,
    'overall_passed': all(targets_met),
    
    'overfitting_analysis': {
        'train_test_da_gap_pp': float(train_test_da_gap),
        'target_gap_pp': 10.0,
        'overfitting_check': 'PASS' if train_test_da_gap < 10 else 'FAIL',
    },
}

with open('training_result.json', 'w') as f:
    json.dump(training_result, f, indent=2, default=str)
print("✓ Saved training_result.json")

print(f"\n{'='*60}")
print("TRAINING COMPLETE")
print(f"{'='*60}")
print(f"Finished: {datetime.now().isoformat()}")
print(f"\nFinal Status:")
print(f"  Ensemble weight: w={best_w:.2f} (XGBoost {best_w*100:.0f}% + Ridge {(1-best_w)*100:.0f}%)")
print(f"  HCDA method: {primary_hcda_method.upper()}")
print(f"  MAE method: {'SCALED' if use_scaled else 'RAW'}")
print(f"  Targets passed: {sum(targets_met)}/4")
if all(targets_met):
    print(f"  ✓✓✓ ALL TARGETS MET ✓✓✓")
else:
    failed = [t for t, m in zip(['DA', 'HCDA', 'MAE', 'Sharpe'], targets_met) if not m]
    print(f"  Improvements needed on: {failed}")